In [ ]:
import datetime as dt
import geopandas as gpd
import os
import fiona
import rasterio.mask
from rasterio.fill import fillnodata
from rasterstats import zonal_stats
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import rasterio
import ogr
import warnings
import json
import pandas as pd
from earthpy import clip
from shapely.geometry import JOIN_STYLE
from geopandas import GeoSeries, GeoDataFrame

import scipy.spatial
from pathlib import Path
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

# Selecting Datasets
Select the workspace, this is the folder that will be used for the outputs. 
NOTE Select an empty folder as all the files will be deleted from the workspace 

You will also have to select the three datasets used in the analysis. These are:
    
1) Administrative boundaries.

2) Agro Maps

In [ ]:
messagebox.showinfo('AGRODEM Creating base grid', 'Output folder')
workspace = filedialog.askdirectory()

messagebox.showinfo('AGRODEM Creating base grid', 'Select the administrative boundaries')
filename_admin = (filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
admin=gpd.read_file(filename_admin)

messagebox.showinfo('AGRODEM Creating base grid', 'Select the agro map')
filename_agro = (filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
agro=gpd.read_file(filename_agro)

# Select the target coordinate system


Find the appropriate target crs from:  https://epsg.io/

In [ ]:
crs = 'EPSG:3037'

# Ensuring vectors are in the same coordinate system

In [ ]:
# reprojecing to target coordinate system written above
def target_crs(vectors,crs,workspace):   
    vectors = vectors.to_crs(crs) 
    vectors.to_file(workspace, driver='ESRI Shapefile')  
    return vectors

agro_pr = agro.to_crs(crs)
admin_pr = admin.to_crs(crs)


# Fix geometries

In [ ]:
def fixgeometries(polygon):
    #creates a valid representation of a given invalid geometry without losing any of the input vertices. 
    fix = polygon.buffer(0.001)
    return fix

# Preparing agro maps

In [ ]:
#fix geometries
fixed_agro = fixgeometries(agro_pr)

#clip agro map to place of interest
clipped_agro = gpd.clip(fixed_agro,admin_pr)